#**SETTING UP THE DATA GENERATORS TO GET THE DATA**

In [3]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from google.colab import drive

##iv3 ün weight değerleri
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
from tensorflow.keras.applications.inception_v3 import InceptionV3

weight_file= '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'


--2020-07-26 14:37:43--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   174MB/s    in 0.5s    

2020-07-26 14:37:43 (174 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [18]:
ls

drive/  sample_data/


In [25]:


from tensorflow.keras.preprocessing.image import ImageDataGenerator
drive.mount('/content/drive')
import os
input_size=(128,128)
test_path=''
test_path='./drive/My Drive/kaggle_test_folder/'
###############DATA GENERATOR#############################
image_generator = ImageDataGenerator(rescale=1./255)
validation_generator = image_generator.flow_from_directory(
    test_path,
    target_size=(128,128),
    batch_size = 256,
    class_mode = None
)







#########################################################################################################



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 10982 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **TRAINING Transfer Learning Model Over Big Data**

> Data has resampled and dehaired.

> In each epoch model fits itself to 5000 samples of the data




In [26]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
model_I3_trained= InceptionV3(input_shape= (input_size[0], input_size[1],3),
                      include_top= False,
                      weights= None
                      )

model_I3_trained.load_weights(weight_file)
for layer in model_I3_trained.layers:
  layer.trainable= False





from tensorflow.keras.optimizers import RMSprop

last_layer= model_I3_trained.get_layer('mixed7')
last_output= last_layer.output

x= layers.Flatten()(last_output)
x= layers.Dense(2048,activation='relu')(x)
x= layers.Dense(1, activation='sigmoid')(x)


model_T= Model(model_I3_trained.input, x)
model_T.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])

model_T.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 63, 63, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 63, 63, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 63, 63, 32)   0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

**LOAD trained transfer learning model**

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [28]:
path = "drive/My Drive/transfer_model/model_transfer_hairless.h5"
model_T.load_weights(path)


In [ ]:
predictions = list()
names = list()
c = model_T.predict_generator(validation_generator)
for i in range (0,len(test_fnames)):
  names.append(test_fnames[i])
  predictions.append(c[i][0])
record_list = [ list(item) for item in list(zip(names, predictions)) ]
import csv
with open("sample.csv", "a") as fp:
    writer = csv.writer(fp)
    writer.writerows(record_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

